# OOP — Session 2 Notes
## Encapsulation, Composition, Dynamic Extension, Polymorphism/Duck Typing, Conversion, Class vs Instance Variables, classmethod/staticmethod, Decorators, Dataclasses, SRP




## 1) Encapsulation
Encapsulation bundles **data (attributes)** and **behavior (methods)** inside a class, and controls how data is accessed/updated.

Python uses conventions:
- `_name` → "protected" by convention (still accessible)
- `__name` → name-mangled "private-ish" (harder to access accidentally)


In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.__balance = balance  # private-ish

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit must be positive")
        self.__balance += amount

    def withdraw(self, amount):
        if amount > self.__balance:
            raise ValueError("Insufficient funds")
        self.__balance -= amount

    def get_balance(self):
        return self.__balance

acct = BankAccount("Ritesh", 100)
acct.deposit(50)
print("Balance:", acct.get_balance())

# direct access is blocked by name mangling
try:
    print(acct.__balance)
except Exception as e:
    print("Error:", type(e).__name__, "-", e)

# technically accessible
print("Mangled access:", acct._BankAccount__balance)


## 2) Composition 
Composition means a class **contains** another object and uses it.

- Inheritance = "is-a" (Dog is an Animal)
- Composition = "has-a" (Car has an Engine)


In [ ]:
class Engine:
    def start(self):
        return "Engine started"

class Car:
    def __init__(self):
        self.engine = Engine()  # composition

    def start(self):
        return self.engine.start()

c = Car()
print(c.start())


## 3) Dynamic Extension + `render()` (Decorator Pattern with Composition)
Your screenshot shows `Text.render()` plus wrapper classes that add behavior without changing `Text`.

This is the **Decorator Pattern** (OOP pattern):
- wrap an object
- forward calls
- add extra behavior


In [ ]:
class Text:
    def render(self):
        return "Hello"

class BoldWrapper:
    def __init__(self, wrapped):
        self.wrapped = wrapped  # wrap another object

    def render(self):
        return f"<b>{self.wrapped.render()}</b>"

class ItalicWrapper:
    def __init__(self, wrapped):
        self.wrapped = wrapped

    def render(self):
        return f"<i>{self.wrapped.render()}</i>"


In [ ]:
simple = Text()

bold = BoldWrapper(simple)
italic = ItalicWrapper(simple)
italic_bold = ItalicWrapper(bold)

print(italic_bold.render())
print(bold.render())
print(italic.render())


In [ ]:
class UnderlineWrapper:
    def __init__(self, wrapped):
        self.wrapped = wrapped

    def render(self):
        return f"<u>{self.wrapped.render()}</u>"

u = UnderlineWrapper(ItalicWrapper(BoldWrapper(Text())))
print(u.render())


## 4) Polymorphism ( Duck Typing)
**Polymorphism**: same method name, different behavior.

**Duck typing**: in Python we often care about behavior, not exact types.
If an object has the method we need, we can use it.


In [ ]:
class Dog:
    def speak(self):
        return "Woof"

class Cat:
    def speak(self):
        return "Meow"

def make_speak(obj):
    # duck typing: assumes obj has .speak()
    print(obj.speak())

make_speak(Dog())
make_speak(Cat())


Duck typing example with `render()`:
Anything that implements `.render()` can be wrapped.

In [ ]:
class CustomMessage:
    def __init__(self, msg):
        self.msg = msg

    def render(self):
        return self.msg

msg = CustomMessage("Hi from duck typing")
print(BoldWrapper(msg).render())


## 5) Implicit vs Explicit Data Conversion

- **Implicit**: Python converts automatically in some numeric operations
- **Explicit**: you force conversion using `int()`, `float()`, `str()`, etc.


In [ ]:
# implicit numeric conversion
x = 5 + 2.0
print(x, type(x))

# explicit conversion
print(int("10") + 5)

try:
    int("12.3")
except Exception as e:
    print(type(e).__name__, "-", e)


## 6) Class variables vs Instance variables

- **Instance variable**: belongs to a specific object (e.g., `self.value`)
- **Class variable**: shared across all instances (e.g., `MyClass.class_var`)


In [ ]:
class MyClass:
    class_var = 100  # class variable

    def __init__(self, value):
        self.value = value  # instance variable

    def show(self):
        print(f"Instance value: {self.value}")
        print(f"Class variable before: {MyClass.class_var}")

        # modify both
        self.value += 10
        MyClass.class_var -= 5

        print(f"Instance value after: {self.value}")
        print(f"Class variable after: {MyClass.class_var}")

obj1 = MyClass(15)
obj1.show()

obj2 = MyClass(1)
print("obj2 instance value:", obj2.value)
print("Shared class_var:", MyClass.class_var)


Common bug:
If you do `self.class_var = ...` you may create an **instance attribute** that hides the class variable.


In [ ]:
obj3 = MyClass(0)
obj3.class_var = 999  # creates an instance attribute (does NOT change MyClass.class_var)

print("obj3.class_var (instance shadow):", obj3.class_var)
print("MyClass.class_var (real shared):", MyClass.class_var)


## 7) Instance methods vs Class methods vs Static methods

### Instance method
- first parameter: `self`
- uses instance data

### Class method (`@classmethod`)
- first parameter: `cls`
- uses/modifies class-level data
- often used for "factory" constructors

### Static method (`@staticmethod`)
- no `self`, no `cls`
- utility/helper function placed inside class namespace


In [ ]:
class User:
    count = 0

    def __init__(self, name):
        self.name = name
        User.count += 1

    # instance method
    def greet(self):
        return f"Hello, {self.name}" 

    # class method
    @classmethod
    def get_count(cls):
        return cls.count

    # classmethod as a factory constructor
    @classmethod
    def from_email(cls, email):
        name = email.split("@")[0]
        return cls(name)

    # static method
    @staticmethod
    def is_valid_name(name):
        return isinstance(name, str) and len(name.strip()) >= 2

u1 = User("Ritesh")
u2 = User.from_email("koya@example.com")

print(u1.greet())
print(u2.greet())
print("Count:", User.get_count())
print("Valid?", User.is_valid_name("A"))
print("Valid?", User.is_valid_name("Alex"))


## 8) Decorators 

### A) Python decorators (`@something`)
Examples: `@classmethod`, `@staticmethod`, `@dataclass`

### B) Decorator Pattern (OOP pattern)
wrapper `render()` example is the **Decorator Pattern**.
It is not the same as Python `@decorators`.


## 9) Dataclasses + `__post_init__` (your "post-init")

`@dataclass` auto-generates:
- `__init__`
- `__repr__`
- comparisons (optional)

`__post_init__` runs after the auto-generated `__init__`.
Use it for validation or normalization.


In [ ]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int

p = Person("Ritesh", 25)
p

In [ ]:
from dataclasses import dataclass

@dataclass
class PersonValidated:
    name: str
    age: int

    def __post_init__(self):
        if self.age < 0:
            raise ValueError("Age cannot be negative")
        self.name = self.name.strip().title()

print(PersonValidated("  ritesh  ", 25))

try:
    PersonValidated("test", -5)
except Exception as e:
    print(type(e).__name__, "-", e)


## 10) How long should a class be? (SRP — Single Responsibility Principle)

**SRP:** A class should have **one responsibility** (one reason to change).

If a class is doing too many unrelated things (formatting, saving, emailing, DB writing), split it.

- `Report` - builds/generates report content
- `ReportSaver` - saves the report to a file


In [ ]:
class Report:
    def __init__(self, title, content):
        self.title = title
        self.content = content

    def generate(self):
        return f"{self.title}\n{self.content}"

class ReportSaver:
    def save_to_file(self, report, filename):
        with open(filename, "w", encoding="utf-8") as f:
            f.write(report.generate())

r = Report("Weekly Update", "Completed: OOP Session 2 notes")
ReportSaver().save_to_file(r, "report.txt")
print("Saved report.txt")
